## 1.  Understand the Problem Statement & Import Packages and Datasets :

In [ ]:
# Import Required Python Packages :

# Scientific and Data Manipulation Libraries :

import numpy as np
import pandas as pd

# Data Viz & Regular Expression Libraries :

import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')

# Scikit-Learn ML Libraries :

from sklearn.preprocessing import *
from sklearn.model_selection import *
from sklearn.metrics import *

# Garbage Collection Libraries :

import gc

# Boosting Algorithm Libraries :

from sklearn.tree                     import DecisionTreeClassifier
from xgboost                          import XGBRegressor
from catboost                         import CatBoostRegressor
from lightgbm                         import LGBMRegressor
from sklearn.ensemble                 import ExtraTreesClassifier, VotingClassifier

# Date & Time Libraries :
from datetime import datetime
import time

# Setting SEED in Numpy to get Reproducible Results :
np.random.seed(0)